In [24]:
import os
import sys
import argparse
import uuid
import io
import random
import time
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import imageio
import json
import pprint
import pathlib
import open3d as o3d
from tqdm import tqdm
import colored_traceback
colored_traceback.add_hook()

import robosuite as suite
import robosuite.utils.camera_utils as cam_utils
from robosuite.wrappers import GymWrapper
suite.macros.IMAGE_CONVENTION = "opencv"    # Set the image convention to opencv so the images are automatically rendered "right side up" when using imageio (which uses opencv convention)

# Get the home directory of the current user
home_dir = os.path.expanduser('~')
sys.path.append(os.path.join(home_dir, "Desktop/ERLab/robot_manipulation/SceneGrasp/"))

from common.utils.nocs_utils import load_depth
from common.utils.misc_utils import convert_realsense_rgb_depth_to_o3d_pcl, get_o3d_pcd_from_np, get_scene_grasp_model_params
from common.utils.scene_grasp_utils import SceneGraspModel, get_final_grasps_from_predictions_np, get_grasp_vis

from robosuite_utils import load_episodes, create_env

In [2]:
class AttrDict(dict):
    """
    This class allows you to access python dictionary elements using dot 
    notation `obj.key` instead of the usual square brackets `obj['key']`.
    """
    __setattr__ = dict.__setitem__
    __getattr__ = dict.__getitem__


def args_type(default):
    if isinstance(default, bool):
        return lambda x: bool(['False', 'True'].index(x))
    if isinstance(default, int):
        return lambda x: float(x) if ('e' in x or '.' in x) else int(x)
    if isinstance(default, pathlib.Path):
        return lambda x: pathlib.Path(x).expanduser()
    return type(default)

In [3]:
def estimate_object_shape(rgb_img_batch, depth_img_batch, camera_intrinsic_mat, img_width=96, img_height=96):
    """
    Estimates the 3d point cloud shape of each object in the scene. 
    Returns a list of shape [N, ]
    """

    # demo_data_path = pathlib.Path("../../SceneGrasp/outreach/demo_data")
    # data_generator = self.get_demo_data_generator(demo_data_path)

    # TOP_K = 200  # TODO: use greedy-nms for top-k to get better distributions!
    # all_gripper_vis = []
    # for pred_idx in range(pred_dp.get_len()):
    #     (pred_grasp_poses_cam_final, pred_grasp_widths, _,) = get_final_grasps_from_predictions_np(
    #         pred_dp.scale_matrices[pred_idx][0, 0],
    #         pred_dp.endpoints,
    #         pred_idx,
    #         pred_dp.pose_matrices[pred_idx],
    #         TOP_K=TOP_K,
    #     )
    #     # print(pred_grasp_poses_cam_final.shape)    # (N=124/200/164, 4, 4)
    #     # print(pred_grasp_widths.shape)    # (N,)

    #     grasp_colors = np.ones((len(pred_grasp_widths), 3)) * [1, 0, 0]    # (N, 3)
    #     all_gripper_vis += [get_grasp_vis(pred_grasp_poses_cam_final, pred_grasp_widths, grasp_colors)]

    # pred_pcls = pred_dp.get_camera_frame_pcls()    # list containing the pcd of each object in the image. Each element is np array of shape (2048, 3)
    # pred_pcls_o3d = []
    # for pred_pcl in pred_pcls:
    #     pred_pcls_o3d.append(get_o3d_pcd_from_np(pred_pcl))    # convert numpy pcd to open3d pcd
    # o3d_pcl = convert_realsense_rgb_depth_to_o3d_pcl(rgb, depth / 1000, self._camera_intrinsic_mat)    # Pointcloud with T (variable) points

        # print(">Showing predicted shapes:")
        # print("stage:0", o3d_pcl)
        # print("stage:1", pred_pcls_o3d)
        # print("stage:2", all_gripper_vis)

    print("Wanna REACH HERE")
    pdb.set_trace()
    
    return o3d_pcl, pred_pcls_o3d

In [2]:
def display_image(img):
    # Display the image
    # cv2.imshow('Image', img)

    # # Wait for a key press and then close the window
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()

    display(img)
    return

In [8]:
def define_config():
    config = AttrDict()

    config.env_name = "Lift"    # [PickPlaceBread, PickPlaceMilk, PickPlaceCereal, Lift, Reach]
    config.robots = "Panda"     # "Sawyer", "UR5e"
    config.controller = "OSC_POSE"
    if config.env_name.startswith("PickPlace"):
        config.horizon = 250
    elif config.env_name == "Lift" or config.env_name == "Reach":
        config.horizon = 100
    else:
        config.horizon = 250

    config.camera_names = ["frontview"]
    # config.camera_names = ["frontview", "agentview", "robot0_eye_in_hand"]

    config.use_camera_obs = True
    config.use_depth_obs = True
    config.use_object_obs = True
    config.use_proprio_obs = True
    config.use_touch_obs = True
    config.use_tactile_obs = False
    config.use_shape_obs = False

    config.img_width = 256
    config.img_height = 256

    config.expert_dir_timestamp = datetime(2024, 3, 25, 16, 20, 24).strftime('%Y%m%dT%H%M%S')
    
    return config

In [9]:
config = define_config()
env = create_env(config, verbose=True)

# load the camera intrinsic matrix
camera_k = {}
if isinstance(config.camera_names, str):
    camera_k[config.camera_names] = cam_utils.get_camera_intrinsic_matrix(env.sim, config.camera_names, config.img_height, config.img_width)
elif isinstance(config.camera_names, list):
    for camera_name in config.camera_names:
        camera_k[camera_name] = cam_utils.get_camera_intrinsic_matrix(env.sim, camera_name, config.img_height, config.img_width)
else:
    raise ValueError("Incorrect config.camera_names")

# if config.env_name == "Lift":
#     policy = LiftPolicy(env)
# elif config.env_name == "Reach":
#     policy = ReachPolicy(env)
# elif config.env_name.startswith("PickPlace"):
#     policy = PickPlacePolicy(env)
# print("Policy object created")

timestamp = config.expert_dir_timestamp
expert_dir = pathlib.Path(f"../expert_data/robosuite/{config.env_name.lower()}/{config.robots.lower()}/{config.controller.lower()}/{timestamp}-{config.horizon}/")
print("Expert directory:", expert_dir)

episodes = load_episodes(expert_dir / "episodes/")
episodes = [val for _, val in episodes.items()]
num_eps = len(episodes)

env.close()

Environment created
0 Var: robot0_joint_pos_cos, shape: (7,), range: (-0.984718328481616, 0.9999996408110812)
1 Var: robot0_joint_pos_sin, shape: (7,), range: (-0.48398395104917447, 0.7069664825132252)
2 Var: robot0_joint_vel, shape: (7,), range: (0.0, 0.0)
3 Var: robot0_eef_pos, shape: (3,), range: (-0.11390058656385811, 0.9988640022208465)
4 Var: robot0_eef_quat, shape: (4,), range: (-0.033086587336056024, 0.997844118038471)
5 Var: robot0_gripper_qpos, shape: (2,), range: (-0.020833, 0.020833)
6 Var: robot0_gripper_qvel, shape: (2,), range: (0.0, 0.0)
7 Var: frontview_image, shape: (256, 256, 3), range: (6, 255)
8 Var: frontview_depth, shape: (256, 256, 1), range: (0.9920311570167542, 0.9972583055496216)
9 Var: robot0_touch, shape: (2,), range: (0.0, 0.0)
10 Var: cube_pos, shape: (3,), range: (0.013955491168617565, 0.8308280673292359)
11 Var: cube_quat, shape: (4,), range: (-0.7262194389151002, 0.687462963760112)
12 Var: cube_to_robot0_eef_pos, shape: (3,), range: (-0.137740582007540

In [10]:
hparams = get_scene_grasp_model_params()
print("Loading model from saved checkpoint: ", hparams.checkpoint)
scene_grasp_model = SceneGraspModel(hparams)

Loading model from saved checkpoint:  ../../SceneGrasp/checkpoints/scene_grasp.ckpt
Using model class from: /Users/saqib/Desktop/ERLab/robot_manipulation/SceneGrasp/simnet/lib/net/models/panoptic_net.py
Restoring from checkpoint: ../../SceneGrasp/checkpoints/scene_grasp.ckpt


In [25]:
rgb_cam_names = [key for key in episodes[0].keys() if "_image" in key]
depth_cam_names = [key for key in episodes[0].keys() if "_depth" in key]

if isinstance(config.camera_names, str):
    rgb_cam_names = [cam_name for cam_name in rgb_cam_names if config.camera_names in cam_name]
    depth_cam_names = [cam_name for cam_name in depth_cam_names if config.camera_names in cam_name]
elif isinstance(config.camera_names, list):
    rgb_temp = [cam_name + "_image" for cam_name in config.camera_names]
    depth_temp = [cam_name + "_depth" for cam_name in config.camera_names] 
    rgb_cam_names = list(set(rgb_temp).intersection(set(rgb_cam_names)))
    depth_cam_names = list(set(depth_temp).intersection(set(depth_cam_names)))

num_imgs_success = 0
num_imgs_fail = 0

for episode in tqdm(episodes, desc="Over episodes"):
    for rgb_cam_name, depth_cam_name in zip(rgb_cam_names, depth_cam_names):
        rgb_imgs = episode[rgb_cam_name]    # numpy array
        depth_imgs = episode[depth_cam_name].squeeze()    # numpy array
        num_imgs = rgb_imgs.shape[0]
        cam_intrinsic_mat = camera_k[rgb_cam_name.rsplit("_image", 1)[0]]

        for rgb_img, depth_img in zip(rgb_imgs, depth_imgs):
            # rgb = cv2.resize(rgb_img_batch[idx], (img_width, img_height))
            # depth = cv2.resize(depth_img_batch[idx], (img_width, img_height))
            # rgb_img = cv2.cvtColor(rgb_img, cv2.COLOR_BGR2RGB)    # Convert the image from BGR to RGB (cv2 reads images in BGR format)
            
#             rgb_img_pil = Image.fromarray(rgb_img)    # Convert the image to a PIL Image object
#             depth_img_pil = Image.fromarray(depth_img)
#             # display(rgb_img_pil)
#             # display(depth_img_pil)
#             # rgb_img_pil.show()
#             # depth_img_pil.show()

            pred_dp = scene_grasp_model.get_predictions(rgb_img, depth_img, cam_intrinsic_mat)
            if pred_dp is None:
                num_imgs_fail += 1
            else:
                num_imgs_success += 1
                all_gripper_vis = []
                for pred_idx in range(pred_dp.get_len()):
                    pred_grasp_poses_cam_final, pred_grasp_widths, _, = get_final_grasps_from_predictions_np(pred_dp.scale_matrices[pred_idx][0, 0],
                                                                                                            pred_dp.endpoints,
                                                                                                            pred_idx,
                                                                                                            pred_dp.pose_matrices[pred_idx],
                                                                                                            TOP_K=200,)
                    grasp_colors = np.ones((len(pred_grasp_widths), 3)) * [1, 0, 0]    # (N, 3)
                    all_gripper_vis += [get_grasp_vis(pred_grasp_poses_cam_final, pred_grasp_widths, grasp_colors)]
                
                pred_pcls = pred_dp.get_camera_frame_pcls()    # list containing the pcd of each object in the image. Each element is np array of shape (2048, 3)
                pred_pcls_o3d = []
                for pred_pcl in pred_pcls:
                    pred_pcls_o3d.append(get_o3d_pcd_from_np(pred_pcl))    # convert numpy pcd to open3d pcd
                o3d_pcl = convert_realsense_rgb_depth_to_o3d_pcl(rgb_img, depth_img / 1000, cam_intrinsic_mat)    # Pointcloud with T (variable) points

                print(">Showing predicted shapes:")
                o3d.visualization.draw([o3d_pcl] + pred_pcls_o3d)  # type:ignore
                
                print(">Showing predicted grasps:")
                o3d.visualization.draw(pred_pcls_o3d + all_gripper_vis)  # type:ignore
                break

print(f"Success ratio: {num_imgs_success / (num_imgs_success + num_imgs_fail)}")

Over episodes:   4%|▍         | 1/24 [00:04<01:38,  4.28s/it]/Users/saqib/Desktop/ERLab/robot_manipulation/SceneGrasp/scene_grasp/scale_shape_grasp_ae/model/auto_encoder_scale_grasp.py:57: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  endpoints["grasp_width_one_hot"] = F.softmax(out_pc[:, :, 4:14])


>Showing predicted shapes:
FEngine (64 bits) created at 0x3b4650000 (threading is enabled)
FEngine resolved backend: OpenGL


[error] GLFW error: Cocoa: Failed to find service port for display


>Showing predicted grasps:


Over episodes:   8%|▊         | 2/24 [00:51<10:55, 29.80s/it]

...

Over episodes:  29%|██▉       | 7/24 [01:08<01:37,  5.76s/it]

>Showing predicted shapes:


Over episodes:  29%|██▉       | 7/24 [01:11<02:53, 10.22s/it]


KeyboardInterrupt: 

: 

In [5]:
rgb_img_batch

array([[[[204, 192, 183],
         [206, 194, 183],
         [201, 191, 181],
         ...,
         [200, 192, 183],
         [202, 194, 185],
         [203, 195, 186]],

        [[199, 189, 180],
         [204, 193, 184],
         [203, 192, 183],
         ...,
         [208, 200, 191],
         [208, 200, 191],
         [209, 201, 192]],

        [[196, 187, 179],
         [204, 194, 184],
         [207, 195, 186],
         ...,
         [189, 180, 171],
         [187, 178, 169],
         [195, 186, 177]],

        ...,

        [[115, 113, 109],
         [114, 113, 109],
         [115, 113, 109],
         ...,
         [114, 112, 109],
         [114, 112, 109],
         [114, 112, 109]],

        [[115, 113, 109],
         [114, 113, 108],
         [114, 112, 108],
         ...,
         [114, 112, 109],
         [114, 112, 109],
         [114, 112, 108]],

        [[114, 112, 108],
         [114, 112, 108],
         [114, 112, 108],
         ...,
         [113, 111, 108],
        

In [6]:
depth_img_batch.max()

0.99684495

In [10]:
os.path.dirname(os.path.dirname("../expert_data/robosuite_expert/PickPlaceBread/Panda/OSC_POSE/20230708T191838-agentview-250/expert_data/"))

'../expert_data/robosuite_expert/PickPlaceBread/Panda/OSC_POSE/20230708T191838-agentview-250'